# Play versus an agent

Play with pretrained agent yourself!

> WARNING This notebook is no longer functional

## Imports

In [ ]:
import sys

if '..' not in sys.path: sys.path.append('..')
from src.dnd.FieldGenerator import FieldGenerator
from src.dnd.game_utils import *
from src.dnd.game_board import GameState
from src.agent.agent import DnDAgent
from src.agent.agent_utils import agent_take_turn
from src.dnd.game_configs import *

## Load an agent & game config

In [ ]:
board_size, game_config = get_2v2_0_config()

checkpoint_path = '../rnd/2v2-1/overnight-1/gens/agent-gen20-11.0i'
agent = DnDAgent.load_agent(checkpoint_path, strip=True, epsilon=0)

## Play a game

The following cell creates a new game

In [ ]:
gen = FieldGenerator(board_size=board_size).load_from_folder('../Tokens')
game, colormap = decorate_game(gen.generate_balanced_game(targetCR=1))

In [ ]:
game_over = False

print_game(game, colormap)
input('Press Enter to start')

while game.get_game_state(game.current_player_id) == GameState.PLAYING:
    if game.current_player_id == 1:
        print_turn_info(agent_take_turn(game, agent, get_turn_info=True))
        print_game(game, colormap)
        continue
    
    while True:
        move_input = input("It's your turn. Enter the desired coordinates to move to `y x`:")
        new_coords = tuple([int(x) for x in move_input.split()])
        attack_input = input('Enter the coordinates of unit you want to attack `y x`, or press Enter to skip attack:')
        
        action = None
        if attack_input != '':
            unit = game.current_unit
            target_unit = game.board[tuple([int(x) for x in attack_input.split()])]
            action = unit.actions[0].instantiate(unit, target_unit)

        try:
            print(new_coords)
            print(action)
            game.check_move_legal(new_coords, raise_on_illegal=True)
            game.check_action_legal(action, raise_on_illegal=True)
            game.move(new_coords)
            game.use_action(action)
            game.finish_turn()
            print_game(game, colormap)
            break
        
        except Exception as e: print(e)